In [155]:
# Carregar dependências

import re
import pandas as pd
from pandas.io.parsers.readers import read_csv

# Lista para receber as linhas do arquivo
try:
    arquivo = read_csv(
        '..\\valida_dados_teste\\input\\input_validaDados_V2.csv', '|')
except IOError:
    pass

# Carregar dicionário
try:
    input_dict = pd.read_csv(
        '..\\valida_dados_teste\\input\\teste_input_dict.csv',
        ";",
        header=None,
        index_col=0,
        squeeze=True,
    ).to_dict()
except IOError:
    pass

C:\Users\jeanB\AppData\Local\Temp\ipykernel_19028\3648594179.py:9: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  arquivo = read_csv(
C:\Users\jeanB\AppData\Local\Temp\ipykernel_19028\3648594179.py:16: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  input_dict = pd.read_csv(
C:\Users\jeanB\AppData\Local\Temp\ipykernel_19028\3648594179.py:16: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  input_dict = pd.read_csv(


In [156]:
# validar correspondência entre colunas do input 'arquivo' e as registradas no input 'input_dict':
colunas = list(arquivo)
colunas = [i.strip() for i in colunas]
noMacth = 0

# Para cada item não encontrado, imprimir
# Se todos os itens for encontrados, imprimir mensagem de sucesso
for i in colunas:
  try:
    input_dict[1][i]
  except KeyError:
    noMacth += 1 
    print('Não encontrado (' + str(i) + ')')
    pass

if noMacth == 0:
  print('Sucesso, dados correspondidos!\nPode avançar!')

Sucesso, dados correspondidos!
Pode avançar!


In [157]:
# Declarar variáveis
linhas = []
countLinha = 0

# Carregar 'linhas = []'
for i in arquivo.values:
    linhas.append(list(i))
    
# Adicionar número da linha
for i in linhas:
    i.append(countLinha)
    countLinha = countLinha + 1

In [158]:
# Carregar 'myDataframe'
myDataframe = []

for i in linhas:
    for j in i:
        if i.index(j) < len(colunas): # Se aplica para todos itens da linha, exceto para o último item
            getColuna = colunas[i.index(j)] # Com o index do item, pegar o nome da coluna
            getDado = str(j).strip().replace("nan", "VAZIO") # Eliminar espaço vazio e aplicar replace
            getColunaLen = int(str(input_dict[2][getColuna])) # Pegar tamanho máximo do campo em 'input_dict'
            getRegexCod = input_dict[3][getColuna] # Pegar código regex em 'input_dict'
            getCampoObrigatorio = input_dict[4][getColuna] # Pegar código de campo obrigatório em 'input_dict'
            moca = "" # Declarar variável que vai receber classificação do dado
            if getDado == 'VAZIO' and getCampoObrigatorio == 'N':
                moca = 'vazio_naoObrigatorio'
            elif getDado == 'VAZIO' and getCampoObrigatorio == 'Y':
                moca = 'vazio_obrigatorio'
            elif len(getDado) > getColunaLen and getCampoObrigatorio == 'Y':
                moca = 'foraDoTamanho_obrigatorio'
            elif len(getDado) > getColunaLen and getCampoObrigatorio == 'N':
                moca = 'foraDoTamanho_naoObrigatorio'
            elif len(getDado) <= getColunaLen and re.match(getRegexCod, getDado):
                moca = 'aprovado'
            else:
                moca = 'reprovado'
            myDataframe.append([str(i[-1]), # 'linha'
                                getColuna, # 'coluna'
                                getDado, # 'dado'
                                input_dict[1][getColuna], # 'colunaTec'
                                getColunaLen, # 'colunaLen'
                                getRegexCod, # 'regexCod'
                                getCampoObrigatorio, # 'campoObrigatorio'
                                moca]) # 'status'

df = pd.DataFrame(myDataframe)  # Converter lista em dataframe

df = df.rename(
    columns={0: "linha", 1: "coluna", 2: "dado", 3: "colunaTec", 4: "colunaLen", 5: "regexCod", 6: "campoObrigatorio", 7: "status"})  # Nomear colunas


df.head(5)

# Para imprimir o df:
# df.to_csv('..\\valida_dados_teste\\output\\df_output.csv', index=False)

,linha,coluna,dado,colunaTec,colunaLen,regexCod,campoObrigatorio,status
0,0,CENTRO_CUSTO,114001001.0,CTT,7,^(\d{7})$,Y,foraDoTamanho_obrigatorio
1,0,DESCRICAO_VERBA,SALARIO NORMAL,VERBA_DESC,20,"(^([A-Z]+[ ]?[A-Z]|[ ]?[A-Z]){1,})$",N,aprovado
2,0,TIPO_VERBA,PROVENTO,VERBA_TIPO,8,^(PROVENTO|DESCONTO)$,Y,aprovado
3,0,SITUACAO_FUNCIONARIO,NORMAL,FUNC_STATUS,8,^(NORMAL|FERIAS|AFASTADO TEMPORARIO|DEMITIDO)$,N,aprovado
4,0,MES,201801,MM,6,^([1-2]{1}[0-9]{3}(0[1-9]|1[0-2]))$,N,aprovado


In [159]:
# Executa contagem de dados por coluna + status
out_df_status = pd.DataFrame({"count": df.groupby(['coluna', 'status']).size()}).reset_index()

# Executa contagem de dados por coluna
totalRegistros = pd.DataFrame({"count": df.groupby(['coluna']).size()}).reset_index()


# Em 'out_df_status', trazer total de dados em 'totalRegistros'
inner_join = pd.merge(out_df_status, 
                    totalRegistros, 
                    on ='coluna', 
                    how ='inner')

# Encontrar o percentual de dados por coluna + dado
inner_join['percent'] = inner_join['count_x'] / inner_join['count_y']

# Formatar dados percentuais
pd.options.display.float_format = (
   "{:.2%}".format)

inner_join

,coluna,status,count_x,count_y,percent
0,CENTRO_CUSTO,foraDoTamanho_obrigatorio,72,100,72.00%
1,CENTRO_CUSTO,vazio_obrigatorio,28,100,28.00%
2,DESCRICAO_VERBA,aprovado,42,100,42.00%
3,DESCRICAO_VERBA,foraDoTamanho_naoObrigatorio,28,100,28.00%
4,DESCRICAO_VERBA,reprovado,30,100,30.00%
5,MES,aprovado,100,100,100.00%
6,REFERENCIA,aprovado,100,100,100.00%
7,SITUACAO_FUNCIONARIO,aprovado,93,100,93.00%
8,SITUACAO_FUNCIONARIO,vazio_naoObrigatorio,7,100,7.00%
9,TIPO_VERBA,aprovado,93,100,93.00%
